In [61]:
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from tqdm import tqdm
from gensim.models import word2vec
from gensim.models import Word2Vec

import plotly
import numpy as np
import plotly.graph_objs as go
from sklearn.manifold import TSNE
import plotly.express as px

In [2]:
data = pd.read_csv('./Data/KhaiiiNounsData_07.csv')

In [4]:
word2vec_data = []

for text in tqdm(data['Token']):
    try:
        text = text.split(' ')
        word2vec_data.append(text)
    except:
        text = []
        word2vec_data.append(text)

100%|██████████| 131845/131845 [00:02<00:00, 46380.56it/s]


In [5]:
len(word2vec_data)

131845

In [6]:
# window: 단어탐색길이
# min_count: 최소빈도
# sg: skip_gram사용

model = word2vec.Word2Vec(word2vec_data, window=10, hs=1, min_count=100, sg=1)

In [7]:
model.save('./word2vec_nouns.model')

In [156]:
model.wv.most_similar(positive=['상승'], topn=20)

[('상승세', 0.8805193901062012),
 ('하락', 0.8128594756126404),
 ('급등', 0.7969217300415039),
 ('상승분', 0.7703857421875),
 ('오름세', 0.7475605607032776),
 ('소폭', 0.7444043159484863),
 ('강세', 0.7281137108802795),
 ('마진', 0.7225346565246582),
 ('가격', 0.7212986946105957),
 ('보합세', 0.7051326632499695),
 ('판가', 0.7027290463447571),
 ('원자재', 0.6906757950782776),
 ('하락세', 0.6868058443069458),
 ('행진', 0.6861264705657959),
 ('요인', 0.6732190847396851),
 ('원자재가격', 0.6721929907798767),
 ('상회', 0.6716650128364563),
 ('수입물가', 0.6716510653495789),
 ('전분기', 0.6632556319236755),
 ('상승폭', 0.6607920527458191)]

In [157]:
text_lst = ['상승', '상승세', '하락', '급등', '상승분', '백신', '화이자', '스트라제네카', '모더', '접종',
            '현대차', '현대자동차', '대차', '기아', '아이오닉', '비트코인', '비트', '코인', '화폐', '이더리움']

category_lst = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3]

In [158]:
vector_lst = []

for i in text_lst:
    vector = model.wv[i]
    vector_lst.append(vector)

In [274]:
tsne = TSNE(n_components = 3, 
            random_state=0,
            perplexity=5,
            learning_rate=10,
            n_iter=1000,
            metric='cosine')

In [292]:
pd.DataFrame(vector_lst, index=text_lst).iloc[::5, :]

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
상승,-0.297319,-0.190322,-0.114350,0.186931,-0.375103,0.617032,-0.040163,-0.040540,0.260830,0.086814,...,0.074826,-0.146137,-0.054299,0.010945,-0.001527,0.238602,0.100324,0.160791,-0.007261,-0.210495
백신,0.202775,-0.284040,0.219696,-0.212530,0.111477,0.234121,0.307840,0.149496,0.333761,-0.120441,...,0.040767,-0.190054,-0.165279,0.239980,-0.255481,-0.244379,0.001071,-0.177604,0.282627,-0.039755
현대차,0.421383,0.005837,0.075752,0.262642,0.087326,-0.241881,0.260981,-0.300168,0.361966,-0.404465,...,-0.503338,0.118216,-0.344860,0.068086,0.587506,-0.063780,-0.107751,-0.248221,0.090941,0.299599
비트코인,0.095208,0.198699,0.149661,0.216107,-0.416759,0.493849,-0.434207,0.288460,0.283511,0.278348,...,-0.376767,-0.237740,-0.073310,0.338847,-0.417170,0.040573,-0.076964,0.143438,-0.570033,-0.467340


In [275]:
word_df = pd.DataFrame(tsne.fit_transform(vector_lst), index=text_lst)
word_df['category'] = category_lst

In [276]:
word_df.iloc[::2, :].drop('category', axis=1)

In [278]:
fig = px.scatter_3d(word_df, x=0, y=1, z=2, color='category', 
                    text = ['{}'.format(i) for i in word_df.index], opacity=0.8)
fig.show()

In [273]:
data = go.Scatter3d(
    x=word_df[0],
    y=word_df[1],
    z=word_df[2],
    text = ['{}'.format(i) for i in word_df.index],
    mode='markers',
    marker=dict(
        size=10,
        color=word_df['category'],
        colorscale='Jet',
        line=dict(
            #color='rgba(217, 217, 217, 0.14)',
            #color='rgb(217, 217, 217)',
            width=0.0
        ),
        opacity=0.8
    )
)

In [257]:
# layout = go.Layout(
#     autosize=False,
#     width=500,
#     height=500,
#     margin=go.Margin(
#         l=50,
#         r=50,
#         b=100,
#         t=100,
#         pad=4
#     ),
#     #paper_bgcolor='#7f7f7f',
#     #plot_bgcolor='#c7c7c7'
# )

In [258]:
fig = go.Figure(data=[data])
fig.show()